<a href="https://colab.research.google.com/github/KorStats/classification-project/blob/main/%EC%A6%9D%EB%B6%84/K%EB%8D%B0%EC%9D%B4%ED%84%B0_%EC%A6%9D%EB%B6%84(after_spell_check).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## 구글드라이브에서 파일 읽어오기

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
from google.colab import drive
drive.mount('/content/gdrive')
!ln -s /content/gdrive/My\ Drive/ /mydrive
!ls /mydrive

Mounted at /content/gdrive
 산업분류자동화		'# df_P_inc.csv'    K_spellcheck.csv    S_ori_model.pt
 B_spellcheck.csv	 E_spellcheck.csv   labels.zip	        val.txt
 classes.txt		 F_ori_model.pt    'My Drive'
'Colab Notebooks'	 H_ori_model.pt     N_spell_check.csv
'답안 작성용 파일.csv'	 J_spellcheck.csv   Q_ori_model.pt


In [90]:
import pandas as pd
df=pd.read_csv('/content/gdrive/MyDrive/K_spellcheck.csv')

In [91]:
df['digit_3'].value_counts()

641    4113
662    1983
651    1715
661    1211
649    1018
653     171
642     165
652       2
Name: digit_3, dtype: int64

In [54]:
# 데이터 증분 코드
import random
import pickle
import re
def get_only_hangul(line):
	parseText= re.compile('/ ^[ㄱ-ㅎㅏ-ㅣ가-힣]*$/').sub('',line)
	return parseText

def synonym_replacement(words, n):
	new_words = words.copy()
	random_word_list = list(set([word for word in words]))
	random.shuffle(random_word_list)
	num_replaced = 0
	for random_word in random_word_list:
		synonyms = get_synonyms(random_word)
		if len(synonyms) >= 1:
			synonym = random.choice(list(synonyms))
			new_words = [synonym if word == random_word else word for word in new_words]
			num_replaced += 1
		if num_replaced >= n:
			break
	if len(new_words) != 0:
		sentence = ' '.join(new_words)
		new_words = sentence.split(" ")
	else:
		new_words = ""
	return new_words

def get_synonyms(word):
	synomyms = []
	try:
		for syn in wordnet[word]:
			for s in syn:
				synomyms.append(s)
	except:
		pass
	return synomyms

def random_deletion(words, p):
	if len(words) == 1:
		return words

	new_words = []
	for word in words:
		r = random.uniform(0, 1)
		if r > p:
			new_words.append(word)
	if len(new_words) == 0:
		rand_int = random.randint(0, len(words)-1)
		return [words[rand_int]]
	return new_words

########################################################################
# Random swap
# Randomly swap two words in the sentence n times
########################################################################
def random_swap(words, n):
	new_words = words.copy()
	for _ in range(n):
		new_words = swap_word(new_words)
	return new_words

def swap_word(new_words):
	random_idx_1 = random.randint(0, len(new_words)-1)
	random_idx_2 = random_idx_1
	counter = 0
	while random_idx_2 == random_idx_1:
		random_idx_2 = random.randint(0, len(new_words)-1)
		counter += 1
		if counter > 3:
			return new_words
	new_words[random_idx_1], new_words[random_idx_2] = new_words[random_idx_2], new_words[random_idx_1]
	return new_words

########################################################################
# Random insertion
# Randomly insert n words into the sentence
########################################################################
def random_insertion(words, n):
	new_words = words.copy()
	for _ in range(n):
		add_word(new_words)	
	return new_words

def add_word(new_words):
	synonyms = []
	counter = 0
	while len(synonyms) < 1:
		if len(new_words) >= 1:
			random_word = new_words[random.randint(0, len(new_words)-1)]
			synonyms = get_synonyms(random_word)
			counter += 1
		else:
			random_word = ""
		if counter >= 10:
			return
		
	random_synonym = synonyms[0]
	random_idx = random.randint(0, len(new_words)-1)
	new_words.insert(random_idx, random_synonym)

def EDA(sentence, alpha_sr=0.1, alpha_ri=0.1, alpha_rs=0.1, p_rd=0.1, num_aug=9):
	sentence = get_only_hangul(sentence)
	words = sentence.split(' ')
	words = [word for word in words if word is not ""]
	num_words = len(words)
	augmented_sentences = []
	num_new_per_technique = int(num_aug/4) + 1
	n_sr = max(1, int(alpha_sr*num_words))
	n_ri = max(1, int(alpha_ri*num_words))
	n_rs = max(1, int(alpha_rs*num_words))
	# sr
	for _ in range(num_new_per_technique):
		a_words = synonym_replacement(words, n_sr)
		augmented_sentences.append(' '.join(a_words))
	# ri
	for _ in range(num_new_per_technique):
		a_words = random_insertion(words, n_ri)
		augmented_sentences.append(' '.join(a_words))
	# rs
	for _ in range(num_new_per_technique):
		a_words = random_swap(words, n_rs)
		augmented_sentences.append(" ".join(a_words))
	# rd
	for _ in range(num_new_per_technique):
		a_words = random_deletion(words, p_rd)
		augmented_sentences.append(" ".join(a_words))
	augmented_sentences = [get_only_hangul(sentence) for sentence in augmented_sentences]
	random.shuffle(augmented_sentences)
	if num_aug >= 1:
		augmented_sentences = augmented_sentences[:num_aug]
	else:
		keep_prob = num_aug / len(augmented_sentences)
		augmented_sentences = [s for s in augmented_sentences if random.uniform(0, 1) < keep_prob]
	augmented_sentences.append(sentence)
	return augmented_sentences

In [92]:
df1=df[df['digit_3']==641] 
df2=df[df['digit_3']==662] 
df3=df[df['digit_3']==651] 
df4=df[df['digit_3']==661] 
df5=df[df['digit_3']==649] 
df6=df[df['digit_3']==653]
df7=df[df['digit_3']==642]
df8=df[df['digit_3']==652] 


df20=pd.concat([df1,df2, df3, df4, df5])
df30=pd.concat([df6, df7])
df40=df8

df20=df20.reset_index(drop=True)
df30=df30.reset_index(drop=True)
df40=df40.reset_index(drop=True)

In [93]:
temp_result=[]
n=0
id_list=[]
digit_1_list=[]
digit_2_list=[]
digit_3_list=[]

for sentence in df20['text']:
  result_list=EDA(sentence, alpha_sr=0.0, alpha_ri=0.0, alpha_rs=0.5, p_rd=0.5, num_aug=2)
  id_list=['p'+str(df20['AI_id'][n]), 'p'+str(df20['AI_id'][n]), 'p'+str(df20['AI_id'][n])]
  digit_1_list=[df20['digit_1'][n], df20['digit_1'][n], df20['digit_1'][n]]
  digit_2_list=[df20['digit_2'][n], df20['digit_2'][n], df20['digit_2'][n]]
  digit_3_list=[df20['digit_3'][n], df20['digit_3'][n], df20['digit_3'][n]]
  data_to_insert={'AI_id' : id_list, 'digit_1' : digit_1_list, 'digit_2' : digit_2_list, 'digit_3': digit_3_list, 'text' : result_list}
  df20_to_insert = pd.DataFrame(data_to_insert)
  df20= df20.append(df20_to_insert, ignore_index=True)
  n=n+1

In [94]:
df20

,AI_id,digit_1,digit_2,digit_3,text
0,id_0000414,K,64,641,신협
1,id_0000631,K,64,641,신협 고객 대상 금융 서비스 금융 서비스
2,id_0000809,K,64,641,신협 고객 대상으로 자금 여수신
3,id_0000923,K,64,641,신협 고객 대상 은행 관련 서비스
4,id_0001132,K,64,641,"신협 고객 대상 자금여, 수신"
...,...,...,...,...,...
40155,pid_0997636,K,64,649,담보대출 및 신용대출
40156,pid_0997636,K,64,649,담보대출 및 신용대출
40157,pid_0998484,K,64,649,사업장에서 은행 유동화채권 부실채권 매입
40158,pid_0998484,K,64,649,사업장에서 은행 유동화채권 부실채권 매입


In [95]:
temp_result=[]
n=0
id_list=[]
digit_1_list=[]
digit_2_list=[]
digit_3_list=[]

for sentence in df30['text']:
  result_list=EDA(sentence, alpha_sr=0.1, alpha_ri=0.1, alpha_rs=0.4, p_rd=0.4, num_aug=39)
  id_list=['p'+str(df30['AI_id'][n]), 'p'+str(df30['AI_id'][n]), 'p'+str(df30['AI_id'][n]), 'p'+str(df30['AI_id'][n]),'p'+str(df30['AI_id'][n]), 'p'+str(df30['AI_id'][n]), 'p'+str(df30['AI_id'][n]), 'p'+str(df30['AI_id'][n]),'p'+str(df30['AI_id'][n]), 'p'+str(df30['AI_id'][n]), 'p'+str(df30['AI_id'][n]), 'p'+str(df30['AI_id'][n]),'p'+str(df30['AI_id'][n]), 'p'+str(df30['AI_id'][n]), 'p'+str(df30['AI_id'][n]), 'p'+str(df30['AI_id'][n]),'p'+str(df30['AI_id'][n]),'p'+str(df30['AI_id'][n]), 'p'+str(df30['AI_id'][n]), 'p'+str(df30['AI_id'][n]), 'p'+str(df30['AI_id'][n]),'p'+str(df30['AI_id'][n]), 'p'+str(df30['AI_id'][n]), 'p'+str(df30['AI_id'][n]), 'p'+str(df30['AI_id'][n]),'p'+str(df30['AI_id'][n]), 'p'+str(df30['AI_id'][n]), 'p'+str(df30['AI_id'][n]), 'p'+str(df30['AI_id'][n]),'p'+str(df30['AI_id'][n]), 'p'+str(df30['AI_id'][n]), 'p'+str(df30['AI_id'][n]), 'p'+str(df30['AI_id'][n]),'p'+str(df30['AI_id'][n]),'p'+str(df30['AI_id'][n]),'p'+str(df30['AI_id'][n]),'p'+str(df30['AI_id'][n]),'p'+str(df30['AI_id'][n]),'p'+str(df30['AI_id'][n]),'p'+str(df30['AI_id'][n])]
  digit_1_list=[df30['digit_1'][n], df30['digit_1'][n], df30['digit_1'][n], df30['digit_1'][n],df30['digit_1'][n], df30['digit_1'][n], df30['digit_1'][n], df30['digit_1'][n],df30['digit_1'][n], df30['digit_1'][n], df30['digit_1'][n], df30['digit_1'][n],df30['digit_1'][n], df30['digit_1'][n], df30['digit_1'][n], df30['digit_1'][n],df30['digit_1'][n],df30['digit_1'][n], df30['digit_1'][n], df30['digit_1'][n], df30['digit_1'][n],df30['digit_1'][n], df30['digit_1'][n], df30['digit_1'][n], df30['digit_1'][n],df30['digit_1'][n], df30['digit_1'][n], df30['digit_1'][n], df30['digit_1'][n],df30['digit_1'][n], df30['digit_1'][n], df30['digit_1'][n], df30['digit_1'][n],df30['digit_1'][n],df30['digit_1'][n],df30['digit_1'][n],df30['digit_1'][n],df30['digit_1'][n],df30['digit_1'][n],df30['digit_1'][n]]
  digit_2_list=[df30['digit_2'][n], df30['digit_2'][n], df30['digit_2'][n], df30['digit_2'][n],df30['digit_2'][n], df30['digit_2'][n], df30['digit_2'][n], df30['digit_2'][n],df30['digit_2'][n], df30['digit_2'][n], df30['digit_2'][n], df30['digit_2'][n],df30['digit_2'][n], df30['digit_2'][n], df30['digit_2'][n], df30['digit_2'][n],df30['digit_2'][n],df30['digit_2'][n], df30['digit_2'][n], df30['digit_2'][n], df30['digit_2'][n],df30['digit_2'][n], df30['digit_2'][n], df30['digit_2'][n], df30['digit_2'][n],df30['digit_2'][n], df30['digit_2'][n], df30['digit_2'][n], df30['digit_2'][n],df30['digit_2'][n], df30['digit_2'][n], df30['digit_2'][n], df30['digit_2'][n],df30['digit_2'][n],df30['digit_2'][n],df30['digit_2'][n],df30['digit_2'][n],df30['digit_2'][n],df30['digit_2'][n],df30['digit_2'][n]]
  digit_3_list=[df30['digit_3'][n], df30['digit_3'][n], df30['digit_3'][n], df30['digit_3'][n],df30['digit_3'][n], df30['digit_3'][n], df30['digit_3'][n], df30['digit_3'][n],df30['digit_3'][n], df30['digit_3'][n], df30['digit_3'][n], df30['digit_3'][n],df30['digit_3'][n], df30['digit_3'][n], df30['digit_3'][n], df30['digit_3'][n],df30['digit_3'][n],df30['digit_3'][n], df30['digit_3'][n], df30['digit_3'][n], df30['digit_3'][n],df30['digit_3'][n], df30['digit_3'][n], df30['digit_3'][n], df30['digit_3'][n],df30['digit_3'][n], df30['digit_3'][n], df30['digit_3'][n], df30['digit_3'][n],df30['digit_3'][n], df30['digit_3'][n], df30['digit_3'][n], df30['digit_3'][n],df30['digit_3'][n],df30['digit_3'][n],df30['digit_3'][n],df30['digit_3'][n],df30['digit_3'][n],df30['digit_3'][n],df30['digit_3'][n]]
  data_to_insert={'AI_id' : id_list, 'digit_1' : digit_1_list, 'digit_2' : digit_2_list, 'digit_3': digit_3_list, 'text' : result_list}
  df30_to_insert = pd.DataFrame(data_to_insert)
  df30= df30.append(df30_to_insert, ignore_index=True)
  n=n+1

In [96]:
df30

,AI_id,digit_1,digit_2,digit_3,text
0,id_0000526,K,65,653,사무실에서 직원들에게 복지 활동
1,id_0007857,K,65,653,사무실에서 사내 복지 기금 기금관리
2,id_0008309,K,65,653,사무실에서 사내 복지 향상을 위해 기금 운용
3,id_0008691,K,65,653,"사무실에서 사내 직원 대상 사내근로복지기금, 복리후생, 상조서비스"
4,id_0008996,K,65,653,시청 직장금고 시청 직원 대상 사내근로복지기금
...,...,...,...,...,...
13771,pid_0986421,K,64,642,"일반인, 대상으로 금융 투자자문 및, 운영"
13772,pid_0986421,K,64,642,"금융 일반인, 투자자문 대상으로 및, 기업 사무실에서 운영"
13773,pid_0986421,K,64,642,"운영 일반인, 사무실에서 기업 금융 투자자문 및, 대상으로"
13774,pid_0986421,K,64,642,"사무실에서 일반인, 기업 대상으로 금융 투자자문 및, 운영"


In [97]:
temp_result=[]
n=0
id_list=[]
digit_1_list=[]
digit_2_list=[]
digit_3_list=[]

for sentence in df40['text']:
  result_list=EDA(sentence, alpha_sr=0.1, alpha_ri=0.1, alpha_rs=0.4, p_rd=0.4, num_aug=79)
  id_list=['p'+str(df40['AI_id'][n]), 'p'+str(df40['AI_id'][n]), 'p'+str(df40['AI_id'][n]), 'p'+str(df40['AI_id'][n]),'p'+str(df40['AI_id'][n]), 'p'+str(df40['AI_id'][n]), 'p'+str(df40['AI_id'][n]), 'p'+str(df40['AI_id'][n]),'p'+str(df40['AI_id'][n]), 'p'+str(df40['AI_id'][n]), 'p'+str(df40['AI_id'][n]), 'p'+str(df40['AI_id'][n]),'p'+str(df40['AI_id'][n]), 'p'+str(df40['AI_id'][n]), 'p'+str(df40['AI_id'][n]), 'p'+str(df40['AI_id'][n]),'p'+str(df40['AI_id'][n]),'p'+str(df40['AI_id'][n]), 'p'+str(df40['AI_id'][n]), 'p'+str(df40['AI_id'][n]), 'p'+str(df40['AI_id'][n]),'p'+str(df40['AI_id'][n]), 'p'+str(df40['AI_id'][n]), 'p'+str(df40['AI_id'][n]), 'p'+str(df40['AI_id'][n]),'p'+str(df40['AI_id'][n]), 'p'+str(df40['AI_id'][n]), 'p'+str(df40['AI_id'][n]), 'p'+str(df40['AI_id'][n]),'p'+str(df40['AI_id'][n]), 'p'+str(df40['AI_id'][n]), 'p'+str(df40['AI_id'][n]), 'p'+str(df40['AI_id'][n]),'p'+str(df40['AI_id'][n]),'p'+str(df40['AI_id'][n]),'p'+str(df40['AI_id'][n]),'p'+str(df40['AI_id'][n]),'p'+str(df40['AI_id'][n]),'p'+str(df40['AI_id'][n]),'p'+str(df40['AI_id'][n]),'p'+str(df40['AI_id'][n]), 'p'+str(df40['AI_id'][n]), 'p'+str(df40['AI_id'][n]), 'p'+str(df40['AI_id'][n]),'p'+str(df40['AI_id'][n]), 'p'+str(df40['AI_id'][n]), 'p'+str(df40['AI_id'][n]), 'p'+str(df40['AI_id'][n]),'p'+str(df40['AI_id'][n]), 'p'+str(df40['AI_id'][n]), 'p'+str(df40['AI_id'][n]), 'p'+str(df40['AI_id'][n]),'p'+str(df40['AI_id'][n]), 'p'+str(df40['AI_id'][n]), 'p'+str(df40['AI_id'][n]), 'p'+str(df40['AI_id'][n]),'p'+str(df40['AI_id'][n]),'p'+str(df40['AI_id'][n]), 'p'+str(df40['AI_id'][n]), 'p'+str(df40['AI_id'][n]), 'p'+str(df40['AI_id'][n]),'p'+str(df40['AI_id'][n]), 'p'+str(df40['AI_id'][n]), 'p'+str(df40['AI_id'][n]), 'p'+str(df40['AI_id'][n]),'p'+str(df40['AI_id'][n]), 'p'+str(df40['AI_id'][n]), 'p'+str(df40['AI_id'][n]), 'p'+str(df40['AI_id'][n]),'p'+str(df40['AI_id'][n]), 'p'+str(df40['AI_id'][n]), 'p'+str(df40['AI_id'][n]), 'p'+str(df40['AI_id'][n]),'p'+str(df40['AI_id'][n]),'p'+str(df40['AI_id'][n]),'p'+str(df40['AI_id'][n]),'p'+str(df40['AI_id'][n]),'p'+str(df40['AI_id'][n]),'p'+str(df40['AI_id'][n]),'p'+str(df40['AI_id'][n])]
  digit_1_list=[df40['digit_1'][n], df40['digit_1'][n], df40['digit_1'][n], df40['digit_1'][n],df40['digit_1'][n], df40['digit_1'][n], df40['digit_1'][n], df40['digit_1'][n],df40['digit_1'][n], df40['digit_1'][n], df40['digit_1'][n], df40['digit_1'][n],df40['digit_1'][n], df40['digit_1'][n], df40['digit_1'][n], df40['digit_1'][n],df40['digit_1'][n],df40['digit_1'][n], df40['digit_1'][n], df40['digit_1'][n], df40['digit_1'][n],df40['digit_1'][n], df40['digit_1'][n], df40['digit_1'][n], df40['digit_1'][n],df40['digit_1'][n], df40['digit_1'][n], df40['digit_1'][n], df40['digit_1'][n],df40['digit_1'][n], df40['digit_1'][n], df40['digit_1'][n], df40['digit_1'][n],df40['digit_1'][n],df40['digit_1'][n],df40['digit_1'][n],df40['digit_1'][n],df40['digit_1'][n],df40['digit_1'][n],df40['digit_1'][n],df40['digit_1'][n], df40['digit_1'][n], df40['digit_1'][n], df40['digit_1'][n],df40['digit_1'][n], df40['digit_1'][n], df40['digit_1'][n], df40['digit_1'][n],df40['digit_1'][n], df40['digit_1'][n], df40['digit_1'][n], df40['digit_1'][n],df40['digit_1'][n], df40['digit_1'][n], df40['digit_1'][n], df40['digit_1'][n],df40['digit_1'][n],df40['digit_1'][n], df40['digit_1'][n], df40['digit_1'][n], df40['digit_1'][n],df40['digit_1'][n], df40['digit_1'][n], df40['digit_1'][n], df40['digit_1'][n],df40['digit_1'][n], df40['digit_1'][n], df40['digit_1'][n], df40['digit_1'][n],df40['digit_1'][n], df40['digit_1'][n], df40['digit_1'][n], df40['digit_1'][n],df40['digit_1'][n],df40['digit_1'][n],df40['digit_1'][n],df40['digit_1'][n],df40['digit_1'][n],df40['digit_1'][n],df40['digit_1'][n]]
  digit_2_list=[df40['digit_2'][n], df40['digit_2'][n], df40['digit_2'][n], df40['digit_2'][n],df40['digit_2'][n], df40['digit_2'][n], df40['digit_2'][n], df40['digit_2'][n],df40['digit_2'][n], df40['digit_2'][n], df40['digit_2'][n], df40['digit_2'][n],df40['digit_2'][n], df40['digit_2'][n], df40['digit_2'][n], df40['digit_2'][n],df40['digit_2'][n],df40['digit_2'][n], df40['digit_2'][n], df40['digit_2'][n], df40['digit_2'][n],df40['digit_2'][n], df40['digit_2'][n], df40['digit_2'][n], df40['digit_2'][n],df40['digit_2'][n], df40['digit_2'][n], df40['digit_2'][n], df40['digit_2'][n],df40['digit_2'][n], df40['digit_2'][n], df40['digit_2'][n], df40['digit_2'][n],df40['digit_2'][n],df40['digit_2'][n],df40['digit_2'][n],df40['digit_2'][n],df40['digit_2'][n],df40['digit_2'][n],df40['digit_2'][n],df40['digit_2'][n], df40['digit_2'][n], df40['digit_2'][n], df40['digit_2'][n],df40['digit_2'][n], df40['digit_2'][n], df40['digit_2'][n], df40['digit_2'][n],df40['digit_2'][n], df40['digit_2'][n], df40['digit_2'][n], df40['digit_2'][n],df40['digit_2'][n], df40['digit_2'][n], df40['digit_2'][n], df40['digit_2'][n],df40['digit_2'][n],df40['digit_2'][n], df40['digit_2'][n], df40['digit_2'][n], df40['digit_2'][n],df40['digit_2'][n], df40['digit_2'][n], df40['digit_2'][n], df40['digit_2'][n],df40['digit_2'][n], df40['digit_2'][n], df40['digit_2'][n], df40['digit_2'][n],df40['digit_2'][n], df40['digit_2'][n], df40['digit_2'][n], df40['digit_2'][n],df40['digit_2'][n],df40['digit_2'][n],df40['digit_2'][n],df40['digit_2'][n],df40['digit_2'][n],df40['digit_2'][n],df40['digit_2'][n]]
  digit_3_list=[df40['digit_3'][n], df40['digit_3'][n], df40['digit_3'][n], df40['digit_3'][n],df40['digit_3'][n], df40['digit_3'][n], df40['digit_3'][n], df40['digit_3'][n],df40['digit_3'][n], df40['digit_3'][n], df40['digit_3'][n], df40['digit_3'][n],df40['digit_3'][n], df40['digit_3'][n], df40['digit_3'][n], df40['digit_3'][n],df40['digit_3'][n],df40['digit_3'][n], df40['digit_3'][n], df40['digit_3'][n], df40['digit_3'][n],df40['digit_3'][n], df40['digit_3'][n], df40['digit_3'][n], df40['digit_3'][n],df40['digit_3'][n], df40['digit_3'][n], df40['digit_3'][n], df40['digit_3'][n],df40['digit_3'][n], df40['digit_3'][n], df40['digit_3'][n], df40['digit_3'][n],df40['digit_3'][n],df40['digit_3'][n],df40['digit_3'][n],df40['digit_3'][n],df40['digit_3'][n],df40['digit_3'][n],df40['digit_3'][n],df40['digit_3'][n], df40['digit_3'][n], df40['digit_3'][n], df40['digit_3'][n],df40['digit_3'][n], df40['digit_3'][n], df40['digit_3'][n], df40['digit_3'][n],df40['digit_3'][n], df40['digit_3'][n], df40['digit_3'][n], df40['digit_3'][n],df40['digit_3'][n], df40['digit_3'][n], df40['digit_3'][n], df40['digit_3'][n],df40['digit_3'][n],df40['digit_3'][n], df40['digit_3'][n], df40['digit_3'][n], df40['digit_3'][n],df40['digit_3'][n], df40['digit_3'][n], df40['digit_3'][n], df40['digit_3'][n],df40['digit_3'][n], df40['digit_3'][n], df40['digit_3'][n], df40['digit_3'][n],df40['digit_3'][n], df40['digit_3'][n], df40['digit_3'][n], df40['digit_3'][n],df40['digit_3'][n],df40['digit_3'][n],df40['digit_3'][n],df40['digit_3'][n],df40['digit_3'][n],df40['digit_3'][n],df40['digit_3'][n]]
  data_to_insert={'AI_id' : id_list, 'digit_1' : digit_1_list, 'digit_2' : digit_2_list, 'digit_3': digit_3_list, 'text' : result_list}
  df40_to_insert = pd.DataFrame(data_to_insert)
  df40= df40.append(df40_to_insert, ignore_index=True)
  n=n+1

In [98]:
df40

,AI_id,digit_1,digit_2,digit_3,text
0,id_0179327,K,65,652,보험사에서 보험사 간에 계약에 의거 재보험 서비스
1,id_0858355,K,65,652,사무실에서 보험사 대상 생명보험 재보험
2,pid_0179327,K,65,652,보험사에서 보험사 간에 계약에 의거 재보험 서비스
3,pid_0179327,K,65,652,보험사에서 보험사 간에 계약에 의거 재보험 서비스
4,pid_0179327,K,65,652,간에 계약에 재보험
...,...,...,...,...,...
157,pid_0858355,K,65,652,사무실에서 보험사 대상 생명보험 재보험
158,pid_0858355,K,65,652,사무실에서 보험사 대상 생명보험 재보험
159,pid_0858355,K,65,652,사무실에서 보험사 대상 생명보험 재보험
160,pid_0858355,K,65,652,보험사


기존 데이터 합치기

In [99]:
df=pd.concat([df20, df30, df40])

In [100]:
df['digit_3'].value_counts()

641    16452
662     7932
653     7011
651     6860
642     6765
661     4844
649     4072
652      162
Name: digit_3, dtype: int64

In [101]:
df.to_csv('df_K_inc.csv', index=False, encoding='utf-8-sig')